### Objective

Pull PL data for Guardinex

In [3]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [16]:

import psycopg2
conn = psycopg2.connect(host = 'localhost',port = 15494,database='sofi_dw',user='rarevalo@sofi.org',password=' ')

!pip install "snowflake-connector-python[pandas]" --quiet

import snowflake.connector

def get_snowflake_connection():
    ctx = snowflake.connector.connect(
        user="<username>",
        password="<password>",
        host="localhost",
        port=1444,
        account="sdm",
        warehouse="DATA_SCIENCE",
        database="DW_PRODUCTION",
        protocol="http"
    )
    return ctx


def run_query(sql):
    with get_snowflake_connection() as ctx:
        with ctx.cursor() as cs:
            cs.execute(sql)
            allthedata = cs.fetch_pandas_all()
            return allthedata
                
pd.set_option('display.max_rows', 500)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### PL_Base

### experian_attributes_json

#### ID_Verification

#### Merge

##### blind_flag

## df_final

##### Guardinex fields

### Stats

In [5]:
!pip install fastparquet -- quiet 
import fastparquet
df = pd.read_parquet("df_final.parquet.gzip", engine = 'fastparquet')

     |████████████████████████████████| 400 kB 8.7 MB/s            
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement quiet (from versions: none)
ERROR: No matching distribution found for quiet


ModuleNotFoundError: No module named 'fastparquet'

In [4]:
def memory_usage_mb(df, *args, **kwargs):
    """Dataframe memory usage in MB."""
    return df.memory_usage(*args, **kwargs).sum()

def reduce_memory_usage(df, deep=True, verbose=True, categories=True):
    # All types that we want to change for "lighter" ones.
    # int8 and float16 are not include because we cannot reduce
    # those data types.
    # float32 is not include because float16 has too low precision.
    numeric2reduce = ["int16", "int32", "int64", "float64"]
    start_mem = 0
    if verbose:
        start_mem = memory_usage_mb(df, deep=deep)

    for col, col_type in df.dtypes.iteritems():
        best_type = None
        if col_type == "object":
            df[col] = df[col].astype("category")
            best_type = "category"
        elif col_type in numeric2reduce:
            downcast = "integer" if "int" in str(col_type) else "float"
            df[col] = pd.to_numeric(df[col], downcast=downcast)
            best_type = df[col].dtype.name
        # Log the conversion performed.
        # if verbose and best_type is not None and best_type != str(col_type):
        # print(f"Column {col} converted from {col_type} to {best_type}")

    if verbose:
        end_mem = memory_usage_mb(df, deep=deep)
        diff_mem = start_mem - end_mem
        percent_mem = 100 * diff_mem / start_mem
        print(
            f"Memory usage decreased from"
            f" {start_mem:.2f}MB to {end_mem:.2f}MB"
            f" ({diff_mem:.2f}MB, {percent_mem:.2f}% reduction)"
        )

In [5]:
import sys
import psutil
# utility functions
def memory_usage():
    print('total memory: {} GB'.format(psutil.virtual_memory()[0]/1000000000))
    print('available memory: {} GB'.format(psutil.virtual_memory()[1]/1000000000))
    print('used memory: {} GB'.format(psutil.virtual_memory()[3]/1000000000))
    print('free memory: {} GB'.format(psutil.virtual_memory()[4]/1000000000))
    print('active memory: {} GB'.format(psutil.virtual_memory()[5]/1000000000))
    print('inactive memory: {} GB'.format(psutil.virtual_memory()[6]/1000000000))
    print('buffers memory: {} GB'.format(psutil.virtual_memory()[7]/1000000000))
    print('cached memory: {} GB'.format(psutil.virtual_memory()[8]/1000000000))
memory_usage()

total memory: 66.714914816 GB
available memory: 49.669763072 GB
used memory: 16.333361152 GB
free memory: 48.955785216 GB
active memory: 16.859963392 GB
inactive memory: 0.323055616 GB
buffers memory: 0.2179072 GB
cached memory: 1.207861248 GB


In [154]:
reduce_memory_usage(df)

MemoryError: Unable to allocate 3.80 GiB for an array with shape (698, 730093) and data type object

In [82]:
df.loc[~df["date_fund"].isnull()].groupby(pd.to_datetime(df['date_submit']).dt.to_period('M')).size()

date_submit
2021-01     7523
2021-02     7183
2021-03    10678
2021-04    11065
2021-05    13960
2021-06    14962
2021-07    14945
2021-08    16843
2021-09    18538
2021-10    19316
2021-11    18307
2021-12    17795
2022-01    25492
2022-02    23372
2022-03    27832
2022-04    28151
2022-05    30910
Freq: M, dtype: int64

In [71]:
df.groupby(pd.to_datetime(df['date_submit']).dt.to_period('M')).size()

date_submit
2021-01    17293
2021-02    15746
2021-03    22087
2021-04    21648
2021-05    27269
2021-06    29475
2021-07    31602
2021-08    35903
2021-09    44559
2021-10    64338
2021-11    44211
2021-12    45634
2022-01    70249
2022-02    58328
2022-03    65642
2022-04    64401
2022-05    71708
Freq: M, dtype: int64

In [73]:
df.loc[df["du_ind"]==1].groupby(pd.to_datetime(df['date_submit']).dt.to_period('M')).size()

date_submit
2021-01    10307
2021-02     9649
2021-03    13933
2021-04    14225
2021-05    17820
2021-06    19147
2021-07    19158
2021-08    21972
2021-09    25454
2021-10    27092
2021-11    24864
2021-12    24189
2022-01    37029
2022-02    33160
2022-03    38602
2022-04    38745
2022-05    42479
Freq: M, dtype: int64

In [76]:
df.loc[~df["date_fund"].isnull()].groupby(pd.to_datetime(df['date_submit']).dt.to_period('M')).size()

date_submit
2021-01     7523
2021-02     7183
2021-03    10678
2021-04    11065
2021-05    13960
2021-06    14962
2021-07    14945
2021-08    16843
2021-09    18538
2021-10    19316
2021-11    18307
2021-12    17795
2022-01    25492
2022-02    23372
2022-03    27832
2022-04    28151
2022-05    30910
Freq: M, dtype: int64

In [9]:
df.confirmed_fraud_flag.value_counts(dropna=False)

0.0    401594
NaN    312268
1.0     16231
Name: confirmed_fraud_flag, dtype: int64

In [10]:
df.du_ind.value_counts()

1    417825
0    312268
Name: du_ind, dtype: int64

In [112]:
df.loc[df["du_ind"]==1,['du_ind','blind_flag']].blind_flag.value_counts(dropna=False)

0    250695
1    167130
Name: blind_flag, dtype: int64

In [113]:
df.loc[df['blind_flag']==1,['confirmed_fraud_flag']].value_counts()

confirmed_fraud_flag
0.0                     160811
1.0                       6319
dtype: int64

In [114]:
df.loc[df['blind_flag']==0,['confirmed_fraud_flag']].value_counts()

confirmed_fraud_flag
0.0                     240783
1.0                       9912
dtype: int64

In [115]:
pt1 = df[['date_submit','confirmed_fraud_flag']].pivot_table(index=[pd.to_datetime(df['date_submit']).dt.to_period('M')], columns =['confirmed_fraud_flag'], aggfunc=len)
pt1.rename(columns ={'date_submit':'confirmed_fraud_flag'},inplace=True)
pt1.columns.rename([None,None],inplace=True)

In [116]:
pt1["0"] = pt1.iloc[:,0]/(pt1.iloc[:,0]+pt1.iloc[:,1])
pt1["1"] = (1-(pt1.iloc[:,0]/(pt1.iloc[:,0]+pt1.iloc[:,1])))

In [117]:
pt1

confirmed_fraud_flag               0         1
                             0.0   1.0                    
date_submit                                               
2021-01                     9999   308  0.970117  0.029883
2021-02                     9384   265  0.972536  0.027464
2021-03                    13636   297  0.978684  0.021316
2021-04                    13867   358  0.974833  0.025167
2021-05                    17403   417  0.976599  0.023401
2021-06                    18604   543  0.971640  0.028360
2021-07                    18571   587  0.969360  0.030640
2021-08                    21244   728  0.966867  0.033133
2021-09                    24191  1263  0.950381  0.049619
2021-10                    25111  1981  0.926879  0.073121
2021-11                    23706  1158  0.953427  0.046573
2021-12                    22804  1385  0.942743  0.057257
2022-01                    34276  2753  0.925653  0.074347
2022-02                    32075  1085  0.967280  0.032720
2022-03                    37666   936  0.975753  0.024247
2022-04                    37805   940  0.975739  0.024261
2022-05                    41252  1227  0.971115  0.028885

#### Du_ind

In [118]:
df.loc[(df["du_ind"]==1) & (df["blind_flag"]==1)].id_decline_flag.value_counts()
df.loc[(df["du_ind"]==1) & (df["blind_flag"]==0)].id_decline_flag .value_counts()

0    163431
1      3699
Name: id_decline_flag, dtype: int64

0    244915
1      5780
Name: id_decline_flag, dtype: int64

In [119]:
df.filter(regex='ever')

,ever_dpd30_co_3m_ind,ever_dpd60_co_6m_ind
0,NaN,NaN
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,NaN,NaN
...,...,...
730098,NaN,NaN
730099,0.0,0.0
730100,NaN,NaN
730101,NaN,NaN


#### Funded_ind

In [120]:
df.loc[(df["funded_ind"]==1),"blind_flag"].value_counts()

0    183780
1    123092
Name: blind_flag, dtype: int64

In [121]:
df['yearmonth'] = pd.to_datetime(df['date_submit']).dt.to_period('M')
# df[pd.to_datetime(df['date_submit']).dt.to_period('M').between(pd.to_datetime(df['date_submit']).dt.to_period('M').min(), pd.to_datetime(df['date_submit']).dt.to_period('M').max()-3)]

In [122]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==1) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-3))].ever_dpd30_co_3m_ind.value_counts()

0.0    87688
1.0      657
Name: ever_dpd30_co_3m_ind, dtype: int64

In [123]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==0) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-3))].ever_dpd30_co_3m_ind.value_counts()

0.0    130706
1.0       922
Name: ever_dpd30_co_3m_ind, dtype: int64

In [126]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==0) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-6))].ever_dpd60_co_6m_ind.value_counts()

0.0    91145
1.0      612
Name: ever_dpd60_co_6m_ind, dtype: int64

In [125]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==1) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-6))].ever_dpd60_co_6m_ind.value_counts()

0.0    61127
1.0      430
Name: ever_dpd60_co_6m_ind, dtype: int64

In [133]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==1) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-5))].straight_roller_ind.value_counts()

0    68421
1      209
Name: straight_roller_ind, dtype: int64

In [130]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==0) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-5))].straight_roller_ind.value_counts()

0    102186
1       299
Name: straight_roller_ind, dtype: int64

In [134]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==0) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-5))].charge_off_at_mob05_deceased.value_counts()

0.0    102466
1.0        15
Name: charge_off_at_mob05_deceased, dtype: int64

In [135]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==1) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-5))].charge_off_at_mob05_deceased.value_counts()

0.0    68616
1.0       12
Name: charge_off_at_mob05_deceased, dtype: int64

In [137]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==1) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-5))].charge_off_at_mob05_fraud.value_counts()

0.0    68522
1.0      106
Name: charge_off_at_mob05_fraud, dtype: int64

In [139]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==0) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-5))].charge_off_at_mob05_fraud.value_counts()

0.0    102318
1.0       163
Name: charge_off_at_mob05_fraud, dtype: int64

In [142]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==0) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-5))].charge_off_at_mob05_bankruptcy.value_counts()

0.0    102466
1.0        15
Name: charge_off_at_mob05_bankruptcy, dtype: int64

In [143]:
df.loc[(df["funded_ind"]==1) & 
       (df["blind_flag"]==1) & 
       (df['yearmonth'].between(df['yearmonth'].min(),df['yearmonth'].max()-5))].charge_off_at_mob05_bankruptcy.value_counts()

0.0    68612
1.0       16
Name: charge_off_at_mob05_bankruptcy, dtype: int64

## Ssn

In [6]:
conn = psycopg2.connect(host = 'localhost',port = 15759,database='sofi_dw',user='isarver@sofi.org',password=' ')

ssn = pd.read_sql("""
select distinct party_id as partyid, cast(value as varchar) as ssn 
from tax b
    where b.snapshot_id is null and b.effective_end_dt is null
    and b.effective_start_dt between date '2021-01-01' and date '2022-05-31';
""", conn)


ssn2 = run_query("""

select 

id as business_application_id, 
b.user_id

from sofi_dw.dwmart.applications_file app
left join sofi_dw.sofidw.product_application_facts paf on app.id = paf.application_id
left join sofi_dw.sofidw.borrowers b on app.applicant_id = b.user_id

where application_type IN ('PL')
and date_submit is not null 
and date_submit  between date('2021-01-01') and date('2022-05-31')

""")

In [7]:
ssn_complete = pd.merge(ssn2, ssn , left_on='USER_ID',right_on = 'partyid')

In [8]:
ssn_complete.columns = ssn_complete.columns.str.lower()
ssn_complete.rename(columns ={'ssn':'ssn2'}, inplace= True )

In [9]:
ssn_complete.head()

,business_application_id,user_id,partyid,ssn2
0,16149041,11390996,11390996,608098038
1,13891903,17012633,17012633,054505049
2,15275824,19486768,19486768,007527493
3,16256839,22200355,22200355,310963228
4,16250240,22200355,22200355,310963228


In [10]:
df2 = pd.merge(df, ssn_complete[["business_application_id","ssn2"]], on= 'business_application_id' ,how= 'left')
df2.ssn2.isnull().sum()


212998

In [11]:
df2.ssn.isnull().sum()

330421

In [19]:
df2.ssn2.isnull().sum()


212998

In [13]:
df.loc[df.ssn.isnull()].groupby(['du_ind']).size()

du_ind
0    239757
1     90664
dtype: int64

In [15]:
df2.loc[df2.ssn2.isnull()].groupby(['du_ind']).size()

du_ind
0    185718
1     27280
dtype: int64

In [145]:
ssn_complete

,BUSINESS_APPLICATION_ID,USER_ID,partyid,ssn
0,16149041,11390996,11390996,608098038
1,13891903,17012633,17012633,054505049
2,15275824,19486768,19486768,007527493
3,16256839,22200355,22200355,310963228
4,16250240,22200355,22200355,310963228
...,...,...,...,...
517401,17221461,17938972,17938972,089743783
517402,17215108,19930993,19930993,600213737
517403,16147847,21350600,21350600,530534008
517404,17214071,21350600,21350600,530534008


In [9]:
!pip install fastparquet --quiet 
import fastparquet
df2 = pd.read_parquet('df_final2.parquet.gzip', engine = 'fastparquet' )

In [10]:
def describe_df(df,floatfmt= '.3f'):
    # Numerical
    print("--" * 20)
    print('Columns:', df.shape[1])
    print('Rows:', df.shape[0])
    print("Memory usage:", (f"({(sys.getsizeof(df) / 1024 ** 2):.2f} Mb)"))

    print("--" * 20)
    print('NUMERICAL VARIABLES:')

    numerical = df.select_dtypes(include=np.number)
    concatenated_numerical = pd.concat([
        round(numerical.isnull().sum() / df.shape[0] * 100, 2).astype(str) + "%",
        numerical.isnull().sum(),
        numerical.count(),
        numerical.min(),
        numerical.mean(),
        numerical.max()
    ], axis=1, keys=["%NULLS", "COUNT_NULLS", "NOT_NULL", 'MIN', 'MEAN', 'MAX'], sort=False).sort_values('COUNT_NULLS',
                                                                                                         ascending=False).reset_index().rename(
        columns={'index': ''})

    t = numerical.mode().T
    t.rename(columns={0: 'MODE'}, inplace=True)
    concatenated_numerical = concatenated_numerical.merge(t, how='left', left_on='', right_on=t.index)
    concatenated_numerical.index = concatenated_numerical.index + 1
    concatenated_numerical = concatenated_numerical.iloc[:, [0, 4, 5, 6, 7, 1, 2, 3]]

    print(tabulate(concatenated_numerical, headers=
    [
        'MIN',
        'MEAN',
        'MAX',
        'MODE',
        "%NULLS",
        "#_NULLS",
        "NOT_NULL",

    ], tablefmt="presto", colalign=("right"), floatfmt=floatfmt))

    # Categorical

    print('-----' * 20)
    print()
    print('CATEGORICAL VARIABLES:')
    categorical = df.select_dtypes(['object','category'])
    if categorical.shape[1] == 0:
        print("No Categorical Variables")
    else:
        concatenated_categorical = pd.concat([

            round(categorical.isnull().sum() / df.shape[0] * 100, 2).astype(str) + "%",
            categorical.isnull().sum(),
            categorical.count()
        ],

            keys=["%NULLS",
                  "COUNT_NULLS",
                  "NOT_NULL"], axis=1, sort=False).sort_values('%NULLS', ascending=False).reset_index().rename(
            columns={'index': ''})

        max_unique = 5
        u_strs = []

        for col in categorical:
            series = categorical.loc[categorical[col].notnull(), col]
            n_unique = series.nunique()
            if n_unique > max_unique:
                u_strs.append(str(n_unique) + ' unique values')
            else:
                u_strs.append(str(series.unique()))

        t = pd.DataFrame(u_strs, categorical.columns)
        t = t.reset_index()
        t = t.rename(columns={'index': '', 0: 'Unique_Values'})
        concatenated_categorical = concatenated_categorical.merge(t, on='')
        concatenated_categorical.index = concatenated_categorical.index + 1

        print(tabulate(concatenated_categorical, headers=
        [
            "%NULLS",
            "#_NULLS",
            "NOT_NULL",
            "Unique_Values"

        ], tablefmt="presto", colalign=("left")))

In [14]:
import sys
from tabulate import tabulate
describe_df(df2[["business_application_id"
  ,"date_submit" 
  ,"first_name"
  ,"last_name"
  ,"email"
  ,"current_phone_number"
  ,"current_addr_line_1"
  ,"current_addr_line_2"
  ,"current_city"
  ,"current_state"
  ,"current_zipcode"
  ,"ssn2"
  ,"date_of_birth"
  ,"channel_clean"
  ,"funded_ind"
  ,"loan_status"
   ,'blind_flag']])

----------------------------------------
Columns: 17
Rows: 730093
Memory usage: (274.61 Mb)
----------------------------------------
NUMERICAL VARIABLES:
    |                         |      MIN |         MEAN |      MAX |         MODE | %NULLS   |   #_NULLS |   NOT_NULL
----+-------------------------+----------+--------------+----------+--------------+----------+-----------+------------
 1  | business_application_id | 10419202 | 15497614.666 | 17765553 | 13362273.000 | 0.0%     |         0 |     730093
 2  | funded_ind              | 0        | 0.420        | 1        |        0.000 | 0.0%     |         0 |     730093
 3  | blind_flag              | 0        | 0.229        | 1        |        0.000 | 0.0%     |         0 |     730093
----------------------------------------------------------------------------------------------------

CATEGORICAL VARIABLES:
    |                      |   %NULLS |   #_NULLS |   NOT_NULL | Unique_Values
----+----------------------+----------+-----------+

In [18]:
run_query("""select * from sofi_dw.dwmart.loans_blended_daily limit 5""")

,LOAN_ID,ID,APPLICANT_ID,COMMONLINE_ID,CURRENT_OWNER,CURRENT_OWNER_NAME,CURRENT_OWNER_TYPE,CURRENT_PRIN,ORIGINAL_PRIN,CUR_INT_RATE,...,INSCHOOL_PENDING_DISBURSEMENT_AMOUNT,INSCHOOL_NUM_ACTUAL_DISBURSEMENTS,INSCHOOL_CUMULATIVE_DISBURSED_PRIN,SCHOOL_TYPE,UG_DEGREE_TYPE,G_DEGREE_TYPE,PL_MONEY_BUNDLE_REDUCTION_PERCENTAGE,ELIGIBILITY_TIER,CURR_VARIABLE_RATE_TYPE,SOFR_RATE_EFFECTIVE_DATE
0,1435878,16658384,22369960,16658384,100438,"Loan Asset Issuer LLC, Series 2021 M-2",whole loan,18634.94,20000.00,13.500,...,NaN,NaN,NaN,None,UNDERGRAD,GRAD,0.0,32,None,None
1,1457910,16662533,22373816,3000010000L6CE01A,300209,Synovus,whole loan,9134.75,9403.05,6.820,...,0.0,0.0,0.0,Not-For-Profit,UNDERGRAD,GRAD,0.0,7,None,None
2,1430708,16660467,22386749,16660467,100441,D. E. Shaw Refraction Investments (Cayman) Lim...,whole loan,14032.18,15000.00,14.750,...,NaN,NaN,NaN,None,UNDERGRAD,GRAD,0.0,34,None,None
3,1431833,16665479,22431765,16665479,100115,SoFi Funding PL XIII LLC,warehouse,15494.94,18000.00,4.865,...,NaN,NaN,NaN,None,UNDERGRAD,GRAD,0.0,1,None,None
4,1442908,16664685,22499995,16664685,100410,Stone Ridge Goldman Sachs,whole loan,13428.28,15000.00,13.500,...,NaN,NaN,NaN,None,UNDERGRAD,GRAD,0.0,32,None,None


In [19]:
%%capture

!pip install pyathena

from pyathena import connect
dl = connect(
    s3_staging_dir='s3://sofi-data-science/exl/',
             region_name='us-west-2')

In [20]:
pd.read_sql("""select * from "datalake_production_kafka_raw"."domain_credit_card_state_credit_card_intermediate_data_v2" limit 5""", dl)

,value,key,offset,kafka_timestamp,kafka_timestamp_type,partition
0,"{data={Application.Birthdate=1980-06-01, KYC.A...",32604c56-0983-4700-b340-bfe47fbfdd8e,939,1605252536021,CreateTime,1
1,"{data={Application.Birthdate=1983-08-26, KYC.A...",e7476b1a-a24f-4d47-9ffb-eca1fdc9e93b,1127,1605288611196,CreateTime,2
2,"{data={Application.Birthdate=1988-02-05, APR=2...",f1f1000c-0e9e-4386-98ec-9ab59cfb8d5d,1128,1605288760016,CreateTime,2
3,"{data={Application.Birthdate=1989-06-13, APR=1...",bb217edf-9783-4d3d-979a-62c5ba51c908,602,1605190642316,CreateTime,2
4,"{data={Application.Birthdate=1960-01-10, APR=2...",f7c04cb3-bb24-49ef-9a78-ea494f96b5e5,603,1605190662549,CreateTime,2
